In [47]:
import os
os.environ['MLFLOW_TRACKING_URI']="https://dagshub.com/Vibhav70/datascienceproject.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME'] = "Vibhav70"
os.environ['MLFLOW_TRACKING_PASSWORD'] = "e50e1763a73241764ceb88f8ae29a61a39843fa3"

In [48]:
import os
os.chdir("D:\mlops\datascienceproject")
%pwd

'D:\\mlops\\datascienceproject'

In [49]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [50]:
from src.datascience.constants import *
from src.datascience.utils.common import *

In [51]:
# this goes in configs/configuration manager
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self)-> ModelEvaluationConfig:
        config=self.config.model_evaluataion
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column= schema.name,
            mlflow_uri="https://dagshub.com/Vibhav70/datascienceproject.mlflow"
        )
        return model_evaluation_config


In [52]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [53]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def eval_metrics(self,actual,pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)
        return rmse,mae, r2
    
    def log_into_mlfow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        print(tracking_url_type_store)

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)

            (rmse,mae,r2) = self.eval_metrics(test_y, predicted_qualities)

            #saving metrics as local
            scores = {"rmse": rmse, "mae":mae, "r2":r2}
            save_json(path=Path(self.config.metric_file_name),data=scores)
            
            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse",rmse)
            mlflow.log_metric("mae",mae)
            mlflow.log_metric("r2",r2)

            #Model registry does not work with file dtore
            if tracking_url_type_store!="file":
                mlflow.sklearn.log_model(model,"model",registered_model_name="elasticnet")
            else:
                mlflow.sklearn.log_model(model,"model")

            


In [54]:

try:
    config=ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation1 = ModelEvaluation(config=model_evaluation_config)
    model_evaluation1.log_into_mlfow()
except Exception as e:
    raise e

[2025-01-02 20:08:04,775: INFO: yaml file: config\config.yaml loaded successfully]
[2025-01-02 20:08:04,777: INFO: yaml file: params.yaml loaded successfully]
[2025-01-02 20:08:04,777: INFO: yaml file: schema.yaml loaded successfully]
[2025-01-02 20:08:04,777: INFO: created directory at:artifacts]
[2025-01-02 20:08:04,783: INFO: created directory at:artifacts/model_evaluation]
https
[2025-01-02 20:08:05,346: INFO: json file saved at artifacts\model_evaluation\metrics.json]


2025/01/02 20:08:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'elasticnet'.
2025/01/02 20:08:15 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: elasticnet, version 1
Created version '1' of model 'elasticnet'.


🏃 View run adventurous-elk-483 at: https://dagshub.com/Vibhav70/datascienceproject.mlflow/#/experiments/0/runs/7e83b7ae13544263b86ec015122b32c4
🧪 View experiment at: https://dagshub.com/Vibhav70/datascienceproject.mlflow/#/experiments/0
